In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile

!pip install plotly
import plotly.express as px

!pip install dataprep
from dataprep.eda import create_report

!pip install h2o
import h2o
from h2o.automl import H2OAutoML

In [ ]:
cancer_df1 = pd.read_csv("C:\\Users\\Lelin\\Downloads\\New\\cervical_cancer.csv")

In [ ]:
cancer_df1.head(10)

In [ ]:
cancer_df1.info()

In [ ]:
cancer_df1.describe()

In [ ]:
cancer_df1 = cancer_df1.replace('?', np.nan)
cancer_df1

In [ ]:
cancer_df1.isnull().sum()

In [ ]:
plt.figure(figsize = (10, 10))
sns.heatmap(cancer_df1.isnull(), yticklabels=False)

In [ ]:
cancer_df1 = cancer_df1.apply(pd.to_numeric)
cancer_df1.info()

In [ ]:
cancer_df1.describe()

In [ ]:
cancer_df1.mean()

In [ ]:
cancer_df1 = cancer_df1.fillna(round(cancer_df1.mean()))
cancer_df1.head()

In [ ]:
h2o.init()

In [ ]:
h2o_df = h2o.H2OFrame(cancer_df1)

h2o_df

In [ ]:
h2o_df.describe()

In [ ]:
train, test = h2o_df.split_frame(ratios=[0.75], seed=121)

In [ ]:
X = train.columns
y = 'ca_cervix'
X.remove(y)

In [ ]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [ ]:
X

In [ ]:
y

In [ ]:
aml = H2OAutoML(balance_classes=True,
                stopping_metric='AUC',    
                seed=121,
                max_models=20,
                exclude_algos = ["StackedEnsemble"])

aml.train(x=X, 
          y=y, 
          training_frame=train)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
exm = aml.explain(test)

In [ ]:
exm = aml.leader.explain(test)

In [ ]:
preds = aml.leader.predict(test)

In [ ]:
df = test.cbind(preds)

df

In [ ]:
df = h2o.as_list(df)

In [ ]:
preds = df[['ca_cervix', 'predict']]
preds

In [ ]:
df.to_csv('df1.csv')
preds.to_csv('preds1.csv')

In [ ]:
!pip install lime
import lime
import lime.lime_tabular

In [ ]:
feature_names = train.columns[0: -1]
feature_names

In [ ]:
train_features_numpy = train[feature_names].as_data_frame().values
train_features_numpy

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(train_features_numpy, 
                                                   feature_names = feature_names,
                                                   mode='classification')

In [ ]:
def findPrediction(instance):
    #instance will be in type of numpy
    df = pd.DataFrame(data = instance, columns = feature_names)
    hf = h2o.H2OFrame(df)
    predictions = aml.leader.predict(hf).as_data_frame()
    predictions = predictions.iloc[:,1:].values
    return predictions

In [ ]:
train

In [ ]:
test

In [ ]:
idx = 10
test_df = test.as_data_frame()
test_numpy = test_df.iloc[idx].values[0:-1]

In [ ]:
print(test_df.shape)

In [ ]:
test_numpy

In [ ]:
exp = explainer.explain_instance(test_numpy, 
                                 findPrediction, 
                                 num_features = len(feature_names))
exp

In [ ]:
exp.show_in_notebook(show_table=True, show_all=True)

In [ ]:
exp.save_to_file('lime11.html')

In [ ]:
idx = 12
test_df = test.as_data_frame()
test_numpy = test_df.iloc[idx].values[0:-1]

exp = explainer.explain_instance(test_numpy, 
                                 findPrediction, 
                                 num_features = len(feature_names))

exp.show_in_notebook(show_table=True, show_all=True)

In [ ]:
exp.save_to_file('lime12.html')

In [ ]:
idx = 15
test_df = test.as_data_frame()
test_numpy = test_df.iloc[idx].values[0:-1]

exp = explainer.explain_instance(test_numpy, 
                                 findPrediction, 
                                 num_features = len(feature_names))

exp.show_in_notebook(show_table=True, show_all=True)

In [ ]:
exp.save_to_file('lime13.html')